### Reference.
Torch-Demos : [digit classifier](https://github.com/torch/demos/blob/master/train-a-digit-classifier/train-on-mnist.lua)

In [1]:
require 'torch'
require 'nn'
require 'optim'
require 'image'
require 'dataset-mnist'

In [2]:
-- Setting Hyperparameters
batchSize = 50
learningRate = 0.05
momentum = 0
numIter = 20

-- Fix Seed
torch.manualSeed(1)

-- Setting worker
torch.setnumthreads(2)
print('<torch> set nb of threads to ' .. torch.getnumthreads())

-- Use Floats
torch.setdefaulttensortype('torch.FloatTensor')

<torch> set nb of threads to 1	


In [3]:
-- Define model to Train
-- On the 10-class Classification Problem
-- Geometry : width and height of input images

classes = {'1', '2' ,'3' ,'4' ,'5' ,'6' ,'7' ,'8' ,'9', '10'}
geometry = {32, 32}

In [4]:
numTrain = 60000
numTest = 10000

-- Create training set and normalize
trainData = mnist.loadTrainSet(numTrain, geometry)
trainData:normalizeGlobal(mean, std)

-- Create test set and normalize
testData = mnist.loadTestSet(numTest, geometry)
testData:normalizeGlobal(mean, std)

<mnist> done	


<mnist> done	


In [5]:
print(trainData)
print(testData)

{
  data : FloatTensor - size: 60000x1x32x32
  normalize : function: 0x0c51c4e8
  labels : ByteTensor - size: 60000
  normalizeGlobal : function: 0x0c51c4b0
  size : function: 0x0c51c590
}
{
  data : FloatTensor - size: 10000x1x32x32
  normalize : function: 0x0c51d378
  labels : ByteTensor - size: 10000
  normalizeGlobal : function: 0x0c51d340
  size : function: 0x0c51d420
}


In [6]:
-- 2 Layer Neural Network
model = nn.Sequential()

model:add(nn.Reshape(32*32))
model:add(nn.Linear(32*32, 2048))
model:add(nn.Tanh())
model:add(nn.Linear(2048, #classes))
model:add(nn.LogSoftMax())

criterion = nn.ClassNLLCriterion()

In [7]:
-- Retrieve parameters and gradients
parameters, gradParameters = model:getParameters()

confusion = optim.ConfusionMatrix(classes)
trainHistory = {}
testHistory = {}

In [8]:
-- Train function
function train(dataset)
    -- epoch tracker
    epoch = epoch or 1

    -- local vars
    local time = sys.clock()

    -- do one epoch
    print('<trainer> on training set:')
    print("<trainer> online epoch # " .. epoch .. ' [batchSize = ' .. batchSize .. ']')
    for t = 1, numTrain, batchSize do
        -- create mini batch
        local inputs = torch.Tensor(batchSize,1,geometry[1],geometry[2])
        local targets = torch.Tensor(batchSize)
        local k = 1
        for i = t, math.min(t+batchSize-1, numTrain) do
            -- load new sample
            local sample = dataset[i]
            local input = sample[1]:clone()
            local _,target = sample[2]:clone():max(1)
            target = target:squeeze()
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end

        -- create closure to evaluate f(X) and df/dX
        local feval = function(x)
            -- just in case:
            collectgarbage()

            -- get new parameters
            if x ~= parameters then
                parameters:copy(x)
            end

            -- reset gradients
            gradParameters:zero()

            -- evaluate function for complete mini batch
            local outputs = model:forward(inputs)
            local f = criterion:forward(outputs, targets)

            -- estimate df/dW
            local df_do = criterion:backward(outputs, targets)
            model:backward(inputs, df_do)

            -- update confusion
            for i = 1,batchSize do
                confusion:add(outputs[i], targets[i])
            end

            -- return f and df/dX
            return f,gradParameters
        end

        -- Perform SGD step:
        sgdState = sgdState or {
            learningRate = learningRate,
            momentum = momentum,
            learningRateDecay = 5e-7
        }
        optim.sgd(feval, parameters, sgdState)

        -- disp progress
        xlua.progress(t, numTrain)
    end
   
    -- time taken
    time = sys.clock() - time
    time = time / numTrain
    print("<trainer> time to learn 1 sample = " .. (time*1000) .. 'ms')

    -- print confusion matrix
    print(confusion)
    table.insert(trainHistory, confusion.totalValid * 100)
    confusion:zero()

    -- next epoch
    epoch = epoch + 1
end

In [9]:
-- test function
function test(dataset)
    -- local vars
    local time = sys.clock()

    -- test over given dataset
    print('<trainer> on testing Set:')
    for t = 1, dataset:size(), batchSize do
        -- disp progress
        xlua.progress(t, dataset:size())

        -- create mini batch
        local inputs = torch.Tensor(batchSize, 1, geometry[1], geometry[2])
        local targets = torch.Tensor(batchSize)
        local k = 1
        for i = t,math.min(t+batchSize-1, dataset:size()) do
            -- load new sample
            local sample = dataset[i]
            local input = sample[1]:clone()
            local _,target = sample[2]:clone():max(1)
            target = target:squeeze()
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end

        -- test samples
        local preds = model:forward(inputs)

        -- confusion:
        for i = 1, batchSize do
            confusion:add(preds[i], targets[i])
        end
    end

    -- timing
    time = sys.clock() - time
    time = time / dataset:size()
    print("<trainer> time to test 1 sample = " .. (time*1000) .. 'ms')

    -- print confusion matrix
    print(confusion)
    table.insert(testHistory, confusion.totalValid * 100)
    confusion:zero()
end

In [ ]:
for i = 1, numIter do
    train(trainData)
    test(testData)
end

In [11]:
function table2Tensor(table)
    tensor = torch.Tensor(#table)
    for i = 1, tensor:size()[1] do
        tensor[i] = table[i]
    end
    return tensor
end

In [12]:
Plot = require 'itorch.Plot'

trainValid = table2Tensor(trainHistory)
testValid = table2Tensor(testHistory)

epochCount = torch.Tensor(numIter)
i = 0

epochCount:apply(function()
    i = i + 1
    return i
end)


-- line plots
plot = Plot():line(epochCount, trainValid, 'green', 'Train')
plot:line(epochCount, testValid, 'blue', 'Test')
plot:legend(true)
plot:title('Train/Test TotalValid'):draw()